In [59]:
# Configs
PROMPT_STYLE = 'AutoCOT1'

raw_path = '../data/1. raw/'
placed_path = '../data/2. placed/'
labeled_path = '../data/3. labelled/'
prompt_path = '../data/4. prompt/'
preprocessed_path = '../data/5. preprocessed/'
compared_path = '../data/6. compared/'

base_prompt_path = prompt_path + f'base_prompt/{PROMPT_STYLE}.txt'
prompt_file_path = prompt_path + f'{PROMPT_STYLE}.txt'
unstructured_data_path = preprocessed_path + f'unstructured_data/{PROMPT_STYLE}.txt'

In [60]:
# Run generator.js
import subprocess

params = ["200", "200"] # Enter the number of data. each prameter indicate the number of small, large map.

subprocess.run(["node", "generator.js"] + params)

CompletedProcess(args=['node', 'generator.js', '200', '200'], returncode=0)

In [61]:
# Run placer.py
import placer

maps = placer.load_maps(path=raw_path)
for map in maps:
    placer.assign_parameters(map, enemy_density=0.05, cohesion=0.3, reward_density=0.01, range_multiplier=2, boss=True)
    placer.modify_map(map, group_min_dist=10, flag_try_count=50, enemy_sparsity=3)
placer.save_maps(maps, path=placed_path)

In [62]:
# Run labeler.py
import labeler

labeler.label(file_num=1, difficulty_curve_interval=5)

In [63]:
# Generate prompt
import prompt_generator

example_count = 10
params = {
    'MAP_SIZE': [20, 20],
    'ROOM_COUNT': 7,
    'ENEMY_COUNT': 2,
    'TREASURE_COUNT': 1
}

with open(base_prompt_path, 'r') as file:
    content = file.read()

    start_prompt, end_prompt = prompt_generator.split_prompts(content)
    example = prompt_generator.load_examples_from_folder(labeled_path, example_count)

    complete_prompt = start_prompt + example + prompt_generator.format_parameters(params) + end_prompt

    with open(prompt_file_path, 'w', encoding='utf-8') as file:
        file.write(complete_prompt)

    print(f'Complete prompt saved to {prompt_file_path}')

Complete prompt saved to ../data/4. prompt/AutoCOT1.txt


In [64]:
# Run GPT
subprocess.run(["python", "unstructured_data_generator.py"])

Traceback (most recent call last):
  File "/home/mintghost/Files/Project/UGRP/src/unstructured_data_generator.py", line 10, in <module>
    response = openai.Completion.create(
  File "/home/mintghost/anaconda3/envs/ugrp_env/lib/python3.10/site-packages/openai/lib/_old_api.py", line 39, in __call__
    raise APIRemovedInV1(symbol=self._symbol)
openai.lib._old_api.APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



CompletedProcess(args=['python', 'unstructured_data_generator.py'], returncode=1)

In [65]:
# Preprocess
subprocess.run(["python", "preprocesser.py"])

Start or end index not found.
Extracted ASCII Art:



CompletedProcess(args=['python', 'preprocesser.py'], returncode=0)

In [66]:
# Run comparator.py
import comparator

comparator.compare(file_num=1)

In [67]:
# Get compared file
import os
import json

path = os.path.join(os.path.dirname(os.getcwd()), "data", "6. compared")

filename_list = [
    filename
    for filename in os.listdir(path)
    if filename.endswith(".json") and os.path.isfile(os.path.join(path, filename))
]

path_list = [os.path.join(path, filename) for filename in filename_list]

compared_list = list()
for file_path in path_list:
    with open(file_path, "r") as file:
        compared_batch = json.load(file)
        for compared in compared_batch["map_list"]:
            compared_list.append(compared)

In [68]:
# Calculate difference
diff_dict = dict()

params = [
    "map_size",
    "density",
    "empty_ratio",
    "exploration_requirement",
    "difficulty_curve",
    "nonlinearity"
]

for param in params:
    diff_dict[param] = list()

for compared in compared_list:
    for param in params:
        try:
            diff_dict[param].append([abs(float(compared["after_params"][param][i]) - float(compared["before_params"][param][i])) for i in range(len(compared["after_params"][param]))])
        except:
            diff_dict[param].append(abs(float(compared["after_params"][param]) - float(compared["before_params"][param])))

In [69]:
# Calculate mean and standard deviation
import numpy as np

mean_dict = dict()
std_dict = dict()

for param in params:
    try:
        mean_dict[param] = np.mean(np.array(diff_dict[param]), axis=0)
        std_dict[param] = np.std(np.array(diff_dict[param]), axis=0)
    except:
        mean_dict[param] = np.mean(diff_dict[param])
        std_dict[param] = np.std(diff_dict[param])

In [70]:
# Print mean and standard deviation
print(mean_dict)
print(std_dict)

{'map_size': array([11., 11.]), 'density': 0.004152762328513162, 'empty_ratio': 0.09090347299468543, 'exploration_requirement': 29.0, 'difficulty_curve': 0.0, 'nonlinearity': 0.21704750661510275}
{'map_size': array([0., 0.]), 'density': 0.0, 'empty_ratio': 0.0, 'exploration_requirement': 0.0, 'difficulty_curve': 0.0, 'nonlinearity': 0.0}
